**Так как наша библиотека совсем пуста на данный момент, заполним ее небольшим количеством наиболее популярных книг с сайта ["100 лучших книг"](https://www.100bestbooks.ru/)**

In [15]:
import requests                      # библиотека для отправки запросов
import numpy as np                   # библиотека матричных и векторных вычислений
import pandas as pd                  # библиотека для работы с табличными данными
import time                          # библиотека для управления временем 
from tqdm import tqdm_notebook       # индикатор выполнения заданного процесса
from fake_useragent import UserAgent # используем для замены агента
from bs4 import BeautifulSoup        # библиотека для обрабортки данных с веб-ресурсов
import json

import warnings
warnings.filterwarnings("ignore")

In [16]:
# итерируемся по страницам сайта
def parsing_pages(page_number):
    """
    Функция для получения данных о книгах (название и автора книги)
    """
    # задаем ссылку на страницу поиска
    page_link = 'https://www.100bestbooks.ru/index.php?page={}'.format(page_number)
    # запрашиваем данные
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    if not response.ok:
        # если сервер нам отказал, сделаем еще пару попыток
        for i in range(3):
            time.sleep(5)
            parsing_pages(page_number)

    # получаем содержимое(контент в формате html) страницы и переводим в объект BeautifulSoup
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    table = soup.find('table', attrs={'class': 'table-rating'})
    return table

In [17]:
# находим по тегу информацию об авторе и названии книги и добавляем их в соответствующие списки
def get_author_title(table):
    objects = table.find_all('span', attrs={'itemprop':'name'})
    authors, book_titles, years = [], [], []
    for i, item in enumerate(objects):  # проходим по всем найденным объектам
        if i % 2 == 0:                  # на четных позициях, начиная с 0 - автор
            authors.append(item.text)   # извлекаем текстовое содержание
        else:                           # на нечетных, начиная с 1 - название книги
            book_titles.append(item.text)
    
    years_links = table.find_all('td', attrs={'class':'vline-year'})
    for i in years_links:
        years.append(i.text)

    return authors, book_titles, years

In [18]:
# пустые списки для сохранения полученных данных
authors_all=[]
titles_all=[]
years_all=[]
for page_number in tqdm_notebook(range(1, 11), desc='Pages'): # для начала возьмем 10 страниц с данного сайта
    # применяем функцию для получения данных
    soup = parsing_pages(page_number)  
    # если вдруг данные не собираются с первого раза
    for i in range(3):
        try:
            # собираем данные
            authors, book_titles, years = get_author_title(soup)
            break
        except:
            # Иначе, пробуем еще несколько раз, пока не закончатся попытки
            time.sleep(3)
            print('Повторим попытку парсинга')
            continue
        # сохраняем данные с каждой странице в соотвествующие списки
    authors_all.extend(authors) 
    titles_all.extend(book_titles)
    years_all.extend(years)

"""
Создаем таблицу с полученными данными: 
id - пронумеруем весь список, начиная с 1
title - сохраняем значения из titles_all
author - сохраняем значения из authors_all
year - к сожалению, данная информация редко размещается заголовочных данных книг, потому что книги переиздаются
status - для начала поставим для всех одинаковый статус "в наличии"
"""
books = pd.DataFrame({'ID': range(1, len(titles_all)+1), 'Название книги': titles_all, 
                                      'Автор книги': authors_all, 'Год издания': years_all, 'Статус': 'в наличии'})
books['Год издания'] = books['Год издания'].apply(lambda x: np.NaN if x=='\xa0' else x)
display(books.head(), books.shape)

Pages:   0%|          | 0/10 [00:00<?, ?it/s]

,ID,Название книги,Автор книги,Год издания,Статус
0,1,Мастер и Маргарита,Михаил Булгаков,1929-1940,в наличии
1,2,Собачье сердце,Михаил Булгаков,1925,в наличии
2,3,Двенадцать стульев,"Илья Ильф, Евгений Петров",1928,в наличии
3,4,Мёртвые души,Николай Гоголь,1842,в наличии
4,5,Граф Монте-Кристо,Александр Дюма,1844-1845,в наличии


(1000, 5)

In [19]:
# сохраним таблицу в текстовый файл
books.to_csv('library.csv', sep=',', index=False)

In [20]:
books = pd.read_csv('library.csv', sep=',')

Можно загрузить сразу в базу данных, например, в PostgreSQL

In [21]:
# import psycopg2
# connection = psycopg2.connect(user="postgres",
#                               host="localhost",
#                               password=password,
#                               port="5432",
#                               database="postgres",
# )

# connection.autocommit = True
# """
# Предварительно необходимо создать таблицу books в базе данных.
# """
# def save_to_db(output_list):
#     with connection.cursor() as cursor:
#         for book in books:
#             cursor.execute("""
#             INSERT INTO books (id, title, author, year, status) VALUES (
#             %s, %s, %s, %s, %s);
#             """,
#             (book['id'], book['title'], book['author'], book['year'], book['status']))